# PyTorch Nerual Network
## 神经网路
神经网络可以通过torch.nn包来构建。  
现在对弈自动梯度(autograd)有一些了解，神经网络是基于自动梯度(autograd)来定义一些模型。  
一个nn.Module包括层和一个方法forward(input)，它会返回输出(output)。  

例如，看一下数字图片识别网络：

![figuire1](https://gitee.com/zyp521/upload_image/raw/master/EhBWqM.png)

这是一个简单的前馈神经网络，它接收输入，让输入一个接着一个的通过一些层，最后给出输出。

### 典型神经网络训练过程
- 定义一个包含可训练参数的神经网络
- 迭代整个输入
- 通过神经网络处理输入
- 计算损失(loss)
- 反向传播梯度到神经网络参数
- 根据梯度调整网络层参数

### 使用
#### 定义一个包含可训练参数的神经网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Create Model
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1,6,5) # input: 1 output: 6  filter:5×5
        self.conv2 = nn.Conv2d(6,16,5) # input: 1 output: 6
        # Fully Connection Layers
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def num_flat_features(self,x):
        """
        number of feature
        
        x:
        """
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def forward(self,x):
        # Max pooling over a (2,2) window
        # 过程说明:输入为(1,1,32,32)的向量，经过第一层卷积网络后得到维度为(1,6,28,28)的向量，然后经过2×2的池化，得到(1,6,14,14)的向量
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If the size is a square you can onlu specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2) # 经过第二次卷积和池化后得到(1,16,5,5)的向量
        # flatted
        x = x.view(-1,self.num_flat_features(x)) #偏平化操作得到(1,16*5*5)=（1，400）的向量
        # activate function
        x = F.relu(self.fc1(x)) # 通过第一层全连接层(1,400)变为(1,120)
        x = F.relu(self.fc2(x)) # 通过第二次全连接层(1,120)变为（1,84）
        x = self.fc3(x) # 通过最后一层全连接层(1,84)变为(1,10)
        return x
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你刚定义了一个前馈函数，然后反向传播函数被自动通过autograd定义了。你可以使用任何张量操作在前馈函数上。  

一个模型可训练的参数可以通过调用net.parameter()返回：

In [2]:
parameters = list(net.parameters())
print(len(parameters))
print(parameters[0].size())  # print conv1's weight matrix dimension
print(parameters[0]) # conv1's weight matrix

10
torch.Size([6, 1, 5, 5])
Parameter containing:
tensor([[[[ 0.0272, -0.1342, -0.0901, -0.0418, -0.0648],
          [ 0.1006,  0.0913,  0.1010,  0.1835, -0.0577],
          [-0.1329, -0.1995, -0.0176,  0.1444, -0.1103],
          [-0.0373,  0.0945,  0.1380,  0.1359, -0.0221],
          [ 0.1764, -0.0775,  0.1039,  0.0238, -0.1366]]],


        [[[-0.1434, -0.1246, -0.1595,  0.0977, -0.1950],
          [ 0.0180,  0.0342,  0.0256, -0.0745, -0.0632],
          [-0.0529,  0.0833,  0.0175,  0.0527,  0.1008],
          [ 0.1544, -0.1421,  0.1826,  0.1198, -0.1565],
          [ 0.0317,  0.0652,  0.0162, -0.1161,  0.0584]]],


        [[[ 0.0654,  0.1471,  0.1411, -0.0976,  0.1472],
          [ 0.1086, -0.0357,  0.1438, -0.0732,  0.1773],
          [ 0.0124,  0.1881,  0.0734, -0.0005, -0.1558],
          [ 0.0602,  0.1759, -0.1495,  0.0255, -0.0949],
          [ 0.1402,  0.1942,  0.0109, -0.0190,  0.0303]]],


        [[[ 0.1996, -0.0101,  0.1107, -0.1305,  0.0150],
          [-0.1123,  0.032

让我们尝试随机生成一个32×32的输入。注意：期望的输入维度是32×32，为了使用在这个网络MINIST数据集上，你需要把数据集中的图片维度改为32×32.

**补充一下对Conv2d的理解**：  
函数格式Conv2d(input,out,filter,stride)，其中m为输入的维度，可以为1，2，3，…… 第二个output为输出的维度，第三个为过滤器大小filter×filter，第四个为步长。  
输入信息:(channel,input,m,m)，channel为输入的通道数，它可以为很多值从1到n，input要和Conv2d中的input保持一致，m×m为输入的二维信息的维度。  
输出信息:(channel,output,m-filter+1,m-filter+1)（步长为1的情况），output和Conv2d保持一致，输出的话，在stride=1的情况下，为m=m-filter+1。如果stride不为1，则要重新考虑。**计算公式为m=(m-filter+1)/stride**。

**对于最大池化层的理解**：  
F.max_pool2d(x,(2,2))，表示输入x向量，使用2×2进行池化。如(1,6,28,28)经过池化后变为(1,6,14,14)，即28/2=14，如果为(4,4)则是28/4=7。

补充一点：如果是用Conv3d的话，输入信息要变为(channel,input,m,m,m)。对于Conv1d的，则变为(channel,input,m)

In [3]:
input  = torch.rand(1,1,32,32)
print(input.size())
out = net(input)
print(out.size())
print(out)

torch.Size([1, 1, 32, 32])
torch.Size([1, 10])
tensor([[-0.0429, -0.0425,  0.0737,  0.0320, -0.1506, -0.0565, -0.0146, -0.0103,
         -0.0058,  0.0468]], grad_fn=<AddmmBackward>)


#### 调用反向传播
把所有参数梯度缓存器置为零，用随机的梯度来进行方向传播。

In [4]:
net.zero_grad() # set zero
out.backward(torch.randn(1,10)) # randn 1-10 use backpropagation

#### 损失函数
**一个损失函数需要一对输入：模型输出和目标，然后计算一个值来评估输出距离目标有多远**。  
有一些不同的损失函数在nn包中。一个简单的损失函数就是nn.MSELoss，这计算了均方误差。

例如：

In [5]:
output = net(input)
target = torch.randn(10) # 随机生成目标值
target = target.view(1,-1) # 扁平化，转换为列向量
print(output,target)
# Loss Function
criterion = nn.MSELoss()
# compute loss
loss = criterion(output,target)

print(loss)

tensor([[-0.0429, -0.0425,  0.0737,  0.0320, -0.1506, -0.0565, -0.0146, -0.0103,
         -0.0058,  0.0468]], grad_fn=<AddmmBackward>) tensor([[ 0.6462, -0.5896,  0.5736,  1.2524, -0.2136,  0.1601, -0.4473, -1.3085,
         -0.8197, -0.7421]])
tensor(0.5721, grad_fn=<MseLossBackward>)


现在，如果你跟随损失到反向传播路径，可以使用它的.grad_fn属性，你将会看到一个这样的计算图：
```
input -> conv2d -> relu -> maxpooled -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> loss
```

所以，当我们调用loss.backward()，整个图都会微分，而且所有的在图中的requires_grad = True 的张量将会让他们的grad张量累积梯度。  

为了掩饰，我们将跟随一下步骤来反向传播。

In [6]:
print(loss.grad_fn) # MESLoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLu

为了实现反向传播损失，我们所有需要做的事情仅仅是使用loss.backward()。你需要清空现存的梯度，要不然梯度将会和现存的梯度累积到一起。

In [7]:
net.zero_grad() # zeroes the gradient buffers of all parameters
# before
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
# backward
loss.backward()
# after
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0038, -0.0029,  0.0021,  0.0113,  0.0006, -0.0010])


#### 更新网络参数
最简单的更新规则是随机梯度下降。

```
weight = weight - learning_rate * gradient
```

我们可以使用python来实现这个规则：

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

尽管如此，如果你使用神经网络，你想使用不同的更新规则，类似于SGD，Nesterov-SGD，Adam，RMSProp等。为了让这可行，我们将建立一个小包：torch.optim实现了所有的方法。使用它非常的简单。

In [16]:
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01) # add parameters and learning rate to optimizer

# training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
# compute loss
loss = criterion(output,target)
loss.backward() # backpropagation

optimizer.step() # gradient update